## Problema
Muitos dos dados provenientes do JSON possuem erros. Além disso, "00_extracao_dados_full" gerou duplicações ao fazer a expansão de uma lista de "assuntos" e uma lista de "movimentos". De forma que os dados da forma como se apresentam agora não são adequados para o cálculo do indicador usado na solução.

Precisamos criar uma visualização dos processos, onde a informação do "processo" seja separada dos "movimentos" associados a ele.

## Solução
Gerar uma visualização dos processos, promovendo uma segunda - mais profunda - melhoria na qualidade dos dados, onde:
- somente os assuntos principais serão considerados;
- remoção de registros com datas de ajuizamento inválidas, estas convertidas na primeira melhoria dos dados;
- remoção de registros com datas de ajuizamento que, embora convertidas corretamente, não fazem sentido;

In [18]:
import pandas as pd 
import numpy as np
from datetime import datetime

local_dados = 'dados/02_tabelas_transformadas/'

In [19]:
def melhorar_qualidade_processo(diretorio):
    """Cria uma visualização apenas com os dados pertinentes ao processo e promove melhoria na qualidade de dados

    Args:
        diretorio (string): Local onde o arquivo processos_full.csv se encontra

    Returns:
        pandas.DataFrame: tabela de processos criada
    """

    # Carga dos processos com seleção de colunas pertinentes
    print('Carregando os dados: ' + local_dados + diretorio + 'processos_full.csv')
    tabela = pd.read_csv(local_dados + diretorio + 'processos_full.csv', low_memory=False, 
                         usecols=['dadosBasicos_numero', 'siglaTribunal', 'grau', 'data_ajuizamento', 'data_insercao', 'data_insercao_str', 'dadosBasicos_codigoLocalidade', 'dadosBasicos_orgaoJulgador_codigoMunicipioIBGE', 'dadosBasicos_orgaoJulgador_codigoOrgao', 'dadosBasicos_orgaoJulgador_instancia', 'dadosBasicos_orgaoJulgador_nomeOrgao',   'data_ajuizamento_str', 'cod_classe', 'descricao_classe', 'descricao_classe_pai', 'cod_assunto', 'descricao_assunto', 'descricao_assunto_pai', 'dadosBasicos_assunto_principal'])
    print('Dados originais')
    print(tabela.shape)
    
    # Filtrar por assunto_principal
    tabela = tabela[tabela.dadosBasicos_assunto_principal==True]
    print('Dados filtrados por assunto principal')
    print(tabela.shape)

    # Remove data_ajuizamento não preenchida
    tabela.dropna(subset=['data_ajuizamento'], inplace=True)
    # Restringe dados a um intervalo de análise desejado [elimina vários valores convertidos errados]
    tabela['xx'] = tabela['data_ajuizamento_str'].str.slice(0, 4)
    tabela = tabela[tabela.xx >= '2005']
    tabela = tabela[tabela.xx < '2021']
    tabela.pop('xx')
    # Converte coluna para data
    tabela['data_ajuizamento'] = pd.to_datetime(tabela['data_ajuizamento'])
    # Remover todos os movimentos posteriores ao dia de hoje... pois não faz sentido
    tabela = tabela[tabela.data_ajuizamento <= datetime.now()]
    print('Dados removidas data_ajuizamento inconsistentes')
    print(tabela.shape)

    # Remover duplicações
    tabela.drop_duplicates(keep='first',inplace=True)
    print('Dados removidas duplicações')
    print(tabela.shape)
    
    return tabela

In [20]:
def criar_tab_processos_full(diretorio):
    """Chama procedimento de melhoria da qualidade do processo, e salva o resultado

    Args:
        diretorio (string): Local onde o arquivo processos_full.csv se encontra
    """
    tabela = melhorar_qualidade_processo(diretorio)

    print('Salvando: ' + local_dados + diretorio + 'tab_processos_full.csv')
    tabela.to_csv(local_dados + diretorio +
                  'tab_processos_full.csv', index=False)
    print()


In [21]:
# Fluxo principal de execução
print("Criação tabProcessosFull iniciada")
criar_tab_processos_full('00_justica_federal/processos-trf1/')
criar_tab_processos_full('00_justica_federal/processos-trf2/')
criar_tab_processos_full('00_justica_federal/processos-trf3/')
criar_tab_processos_full('00_justica_federal/processos-trf4/')
criar_tab_processos_full('00_justica_federal/processos-trf5/')
print("Finalizada criação tabProcessosFull!!")

Criação tabProcessosFull iniciada
Carregando os dados: dados/02_tabelas_transformadas/00_justica_federal/processos-trf1/processos_full.csv
Dados originais
(3018472, 19)
Dados filtrados por assunto principal
(2577467, 19)
Dados removidas data_ajuizamento inconsistentes
(1661694, 19)
Dados removidas duplicações
(19926, 19)
Salvando: dados/02_tabelas_transformadas/00_justica_federal/processos-trf1/tab_processos_full.csv

Carregando os dados: dados/02_tabelas_transformadas/00_justica_federal/processos-trf2/processos_full.csv
Dados originais
(118416, 19)
Dados filtrados por assunto principal
(117806, 19)
Dados removidas data_ajuizamento inconsistentes
(68391, 19)
Dados removidas duplicações
(33542, 19)
Salvando: dados/02_tabelas_transformadas/00_justica_federal/processos-trf2/tab_processos_full.csv

Carregando os dados: dados/02_tabelas_transformadas/00_justica_federal/processos-trf3/processos_full.csv
Dados originais
(1955672, 19)
Dados filtrados por assunto principal
(825049, 19)
Dados re